In [1]:
from plyfile import PlyData
import jax 
import jax.numpy as jnp 
from jax import grad, jacfwd, vmap, jit
import polyscope as ps

from jaxlie import SO3, SE3
from jax.flatten_util import ravel_pytree

import numpy as np
import open3d as o3d 
import trimesh as tm

import meshcat
from meshcat import Visualizer
import meshcat.geometry as mc_geom
import meshcat.transformations as mc_trans

from kernel import create_kernel_matrix, RBF
import jaxopt

import time

objc[35288]: Class GLFWWindowDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x14ba96e90) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x167df6db0). One of the two will be used. Which one is undefined.
objc[35288]: Class GLFWApplicationDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x14ba96e68) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x167df6e28). One of the two will be used. Which one is undefined.
objc[35288]: Class GLFWContentView is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x14ba96ee0) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x167df6e50). One of the two will be used. Which one is undefin

In [3]:
# dataset = o3d.data.BunnyMesh()
# plydata = PlyData.read(dataset.path)
# plydata = PlyData.read('../assets/sphere.ply')
plydata = PlyData.read('../assets/submarine.ply')
vertices = np.vstack((
    plydata['vertex']['x'],
    plydata['vertex']['y'],
    plydata['vertex']['z']
)).T

# _min_pnt = vertices.min(axis=0)
_max_pnt = vertices.max()
# _mid_pnt = vertices.mean(axis=0)
vertices = vertices/_max_pnt

# vertices = (vertices - _mid_pnt)/(_max_pnt-_min_pnt)

faces = np.array(np.vstack(plydata['face']['vertex_indices']))

mesh = tm.Trimesh(vertices=vertices, faces=faces)

# ds_mesh = mesh
ds_mesh = mesh.simplify_quadric_decimation(int(mesh.vertices.shape[0]/10))


# ds_mesh = tm.primitives.Box()
# ds_mesh = ds_mesh.convex_hull

num_points = 1000  # Change this number based on your requirement
points, face_indices = tm.sample.sample_surface(ds_mesh, num_points)

# Get the normals for each sampled point
normals = ds_mesh.face_normals[face_indices]

In [4]:
# XP = jnp.hstack([-ds_mesh.vertex_normals, ds_mesh.vertices + ds_mesh.vertex_normals*0.2])
XP = jnp.hstack([-normals, points + normals*0.2])

X  = jnp.zeros((100,6))

I0000 00:00:1723583369.499610       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [5]:
X_samples =  points + normals*0.2
P_XI      = jnp.ones(X_samples.shape[0])
P_XI = P_XI/np.sum(P_XI)
h = 0.1
args = {'h' : h, 'vertices' : ds_mesh.vertices, 'normals' : ds_mesh.vertex_normals}

In [6]:
# v1 = np.array([0., 0., 1.])
# def phi(x, xp, h=0.01):
#     w, v        = jnp.split(x,2)
#     norm, root  = jnp.split(xp, 2)
#     _R = SO3.exp(w)
#     return jnp.exp(
#         -(jnp.sum((norm+_R@v1)**2)+ np.sum((norm+root-v)**2))/h
#     )

def RBF(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    return jnp.exp(
        -jnp.sum((root-v)**2)/h #- *jnp.sum((norm-w)**2)/h
    ) 
    # * jnp.exp(
    #     -1e-3*jnp.sum((norm-w)**2)/h
    # )
    # return jnp.exp(
    #     -jnp.sum((x-xp)**2)/h
    # )
def camera_view_penalty(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    return jnp.exp(
        -jnp.sum((root-v)**2)/h #- 1e-1*jnp.sum((norm-w)**2)
    ) * jnp.exp(
        -1e-1*jnp.sum((norm-w)**2)
    )

In [7]:
KernelMatrix = create_kernel_matrix(RBF)

camera_view_matrix = create_kernel_matrix(camera_view_penalty)

x0 =  X_samples.min(axis=0)
xf =  X_samples.max(axis=0)
T = 100
X_init = np.linspace(x0, xf, num=T, endpoint=True)
V_init = np.linspace(x0, xf, num=T, endpoint=True)

X = jnp.hstack([V_init, X_init])

sol, unflatten_X = ravel_pytree(X)
bounds = (-np.inf * np.ones_like(sol), np.inf * np.ones_like(sol))

def ergodic_mmd(flat_X, args):
    X_init = unflatten_X(flat_X)
    w, v        = jnp.split(X_init,2, axis=1)

    T = X_init.shape[0]
    h = args['h']
    view_matrix = camera_view_matrix(X_init, XP, h)
    # x0 = args['x0']
    # vertices = args['vertices']
    # norms    = args['normals']
    # XP = jnp.hstack([norms, vertices])
    # P_XI      = jnp.sum(phi_vmapped(X_init, XP, h), axis=1)
    return np.sum(KernelMatrix(X_init, X_init, h))/(T**2) \
            - 2 * np.sum(P_XI @ KernelMatrix(X_init, XP, h))/T \
            + np.mean((v[1:]-v[:-1])**2) - np.sum(P_XI@view_matrix)/T
            # + np.sum((X_init[0]-x0)**2)

solver = jaxopt.ProjectedGradient(fun=ergodic_mmd, projection=jaxopt.projection.projection_box, tol=1e-6)
solver_state = solver.init_state(sol, hyperparams_proj=bounds)
# sol = solver.run(init_params=sol, 
#                         hyperparams_proj=bounds,  
#                         args=args).params

X = unflatten_X(sol)

In [8]:
ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", ds_mesh.vertices, ds_mesh.faces)
ps_bunny_pcl = ps.register_point_cloud("bunny points", points+normals*0.2)


ps_bunny_mesh.set_transparency(0.8)
ps_traj         = ps.register_curve_network("trajectory", X_init * 1.25, edges="line")

for _ in range(1000):
    (sol, solver_state) = solver.update(
                        params=sol, 
                        state=solver_state, 
                        hyperparams_proj=bounds, 
                                args=args)
    X = unflatten_X(sol)
    ps_traj.update_node_positions(X[:,3:])
    ps_traj.add_vector_quantity("vec img", X[:,:3], enabled=True)
    ps.frame_tick()

    time.sleep(0.001)

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 88


In [8]:
ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", ds_mesh.vertices, ds_mesh.faces)
ps_bunny_mesh.set_transparency(0.8)
ps_traj         = ps.register_curve_network("trajectory", X_init * 1.25, edges="line")

# ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, defined_on='vertices', cmap='blues')

ps_traj.update_node_positions(X[:,3:])
ps_traj.add_vector_quantity("vec img", X[:,:3], enabled=True)


for i,_xi in enumerate(X): 
    _dir, _root =  np.split(_xi, 2)
    intrinsics = ps.CameraIntrinsics(fov_vertical_deg=60, aspect=2)

    extrinsics = ps.CameraExtrinsics(root=np.array(_root), look_dir=np.array(_dir), up_dir=(0.,1.,0.))
    params = ps.CameraParameters(intrinsics, extrinsics)
    cam = ps.register_camera_view('cam{}'.format(i), params)
ps.show()

KeyboardInterrupt: 

In [ ]:
def lie_RBF(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    _R = SO3.exp(w)
    return jnp.exp(
        -(jnp.sum((norm+_R@v1)**2) + np.sum((norm+root-v)**2))/h
    )

def create_kernel_matrix(kernel):
    return vmap(vmap(kernel, in_axes=(0, None, None)), in_axes=(None, 0, None))